#     RECURRENT NEURAL NETWORK ' LSTM ' MODEL

## PART I: DATA PRE - PROCESSING

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# importing libraries
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

#Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation
import time

#Plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='Pamela25', api_key='NUbHbkegzXVRKvSlLjqE')

#calculate rmse
from math import sqrt
from sklearn.metrics import mean_squared_error



Using TensorFlow backend.


In [3]:
# Loading the whole dataset and setting Date column as an index

dataset = pd.read_csv(r'C:\Users\Pamela\Desktop\PROJECT\DATASET NASDAQ 100\nasdaq100_joined_closes.csv', index_col=0)

In [4]:
# To check the the dataset from what 'date' to 'date
dataset.head()
dataset.shape

,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AAL,AMGN,...,VRSN,VRSK,VRTX,WBA,WDC,WLTW,WDAY,WYNN,XEL,XLNX
Date,,,,,,,,,,,,,,,,,,,,,
04/01/2010,10.193225,37.090000,9.70,24.125000,18.50,313.688690,311.349976,133.899994,4.557168,48.350658,...,21.007998,29.792044,44.240002,30.470020,37.029556,63.170490,NaN,46.595161,14.790739,20.349150
05/01/2010,10.211267,37.700001,9.71,23.780001,18.01,312.307312,309.978882,134.690002,5.073073,47.931801,...,21.228777,29.991653,42.779999,30.224958,38.011318,63.028927,NaN,49.429043,14.615329,20.092579
06/01/2010,10.157144,37.619999,9.57,23.840000,17.48,304.434448,302.164703,132.250000,4.862889,47.571617,...,20.999506,30.380898,42.029999,29.996222,38.044056,64.043610,NaN,48.780678,14.643392,19.956280
07/01/2010,9.913588,36.889999,9.47,23.930000,17.43,297.347351,295.130463,130.000000,5.006195,47.136017,...,20.710794,30.330994,41.500000,30.175928,37.299545,63.854847,NaN,49.822430,14.580243,19.755833
08/01/2010,9.832404,36.689999,9.43,23.975000,17.66,301.311310,299.064880,133.520004,4.910658,47.554852,...,20.872135,29.941751,40.669998,30.216782,37.487709,63.784065,NaN,49.465473,14.587262,20.044474


(2335, 100)

### Selecting of company :

In [5]:
# In this case study Amazon 'AMZN' company is selected: 

dataset_1 = dataset.iloc[0:,7:8]
dataset_1.head()
dataset_1.shape

,AMZN
Date,
04/01/2010,133.899994
05/01/2010,134.690002
06/01/2010,132.250000
07/01/2010,130.000000
08/01/2010,133.520004


(2335, 1)

###### DATASET VISUALIZATION

In [6]:
trace_one = go.Scatter(
      x = dataset_1.index,
      y = dataset_1['AMZN'],
   name = "Amazon Adj Close",
   line = dict(color='#17BECF'))

data = [trace_one]

layout = dict(title = 'NASQA100 - AMAZON',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Adj Close'))

fig = dict(data=data, layout=layout)
py.iplot(fig, filename = 'LSTM')

### Final dataset from 2015

In [7]:
dataset_f = dataset_1.iloc[1258:]

In [8]:
dataset_f.head()
dataset_f.shape

,AMZN
Date,
02/01/2015,308.519989
05/01/2015,302.190002
06/01/2015,295.290008
07/01/2015,298.420013
08/01/2015,300.459992


(1077, 1)

In [9]:
#Dividing the dataset between training and testing datasets and selecting our target which is 

training_set = dataset_f.iloc[0:1063].values
testing_set = dataset_f.iloc [1063: ].values


In [10]:
# Exploring and exporting 'training_set'
type (training_set)
training_set.shape
#training_set.

numpy.ndarray

(1063, 1)

In [11]:
# Exploring 'testing_set'
type (testing_set)
testing_set.shape
testing_set


numpy.ndarray

(14, 1)

array([[1783.76001 ],
       [1765.699951],
       [1773.420044],
       [1780.75    ],
       [1814.189941],
       [1813.97998 ],
       [1820.699951],
       [1818.859985],
       [1837.280029],
       [1849.859985],
       [1835.839966],
       [1847.329956],
       [1844.069946],
       [1843.060059]])

### Feature Scaling between range 0 and 1

In [12]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)

In [13]:
# To check the data in 'training_set_scaled'
training_set_scaled.shape
type(training_set_scaled)


(1063, 1)

numpy.ndarray

### Creating data with 60 Timesteps 

In [14]:
# Creating a data structure with 60 timesteps and 1 output from training_set
X_train = []
y_train = []
for i in range (60, len(training_set)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])

# converting the lists 'X_train' and 'y_train' in numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)


In [15]:
# To check the data in 'X_train'
type (X_train)
X_train.shape


numpy.ndarray

(1003, 60)

In [16]:
# To check the data in 'y_train'
type (y_train)
y_train.shape


numpy.ndarray

(1003,)

In [17]:
# Reshaping --> in  (Batch size, timesteps, input_dim)
X_train_R = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train_R.shape
type(X_train_R)

(1003, 60, 1)

numpy.ndarray

## Part II : Building the RNN network

In [18]:
# create and fit the LSTM network
model_regressor = Sequential()

#First LSTM layer with 50 neurons as a variable parameter and dropout regularisation to control overfitting
model_regressor.add(LSTM(units=150, return_sequences=True, input_shape = (X_train_R.shape[1], 1)))
model_regressor.add(Dropout(0.2))

#Hidden layer LSTM layer and Dropout regularisation to control overfitting
model_regressor.add(LSTM(units = 150, return_sequences=False))
model_regressor.add(Dropout(0.2)) 

## Output layer and activation function
model_regressor.add(Dense(units=1))
model_regressor.add(Activation('linear'))

# Compiling the RNN
model_regressor.compile(optimizer='adam', loss='mean_squared_error', metrics= ['mae', 'acc'])

# Fit the neural network
model_regressor.fit(X_train_R, y_train, epochs=18, batch_size=128)


Epoch 1/18
1003/1003 [==============================] - 12s 12ms/step - loss: 0.0699 - mean_absolute_error: 0.1872 - acc: 0.0000e+00
Epoch 2/18
1003/1003 [==============================] - 9s 9ms/step - loss: 0.0159 - mean_absolute_error: 0.0911 - acc: 9.9701e-04
Epoch 3/18
1003/1003 [==============================] - 8s 8ms/step - loss: 0.0064 - mean_absolute_error: 0.0707 - acc: 9.9701e-04
Epoch 4/18
1003/1003 [==============================] - 8s 8ms/step - loss: 0.0038 - mean_absolute_error: 0.0445 - acc: 9.9701e-04
Epoch 5/18
1003/1003 [==============================] - 8s 8ms/step - loss: 0.0023 - mean_absolute_error: 0.0362 - acc: 9.9701e-04
Epoch 6/18
1003/1003 [==============================] - 8s 8ms/step - loss: 0.0017 - mean_absolute_error: 0.0282 - acc: 9.9701e-04
Epoch 7/18
1003/1003 [==============================] - 8s 8ms/step - loss: 0.0016 - mean_absolute_error: 0.0278 - acc: 9.9701e-04
Epoch 8/18
1003/1003 [==============================] - 9s 9ms/step - loss: 0.001

## Part III: perform the predictions 

In [19]:
# Creating the inputs

inputs = dataset_f[len(dataset_f) - len(testing_set) - 60:].values

# To check the inputs numpy array
type(inputs)
inputs.shape
len(inputs)
#inputs.head()
print (inputs)

numpy.ndarray

(74, 1)

74

[[1461.640015]
 [1478.02002 ]
 [1501.969971]
 [1539.130005]
 [1500.280029]
 [1575.390015]
 [1629.51001 ]
 [1656.579956]
 [1659.420044]
 [1656.219971]
 [1640.560059]
 [1617.209961]
 [1674.560059]
 [1683.780029]
 [1693.219971]
 [1696.199951]
 [1632.170044]
 [1640.02002 ]
 [1654.930054]
 [1670.569946]
 [1637.890015]
 [1593.880005]
 [1670.430054]
 [1718.72998 ]
 [1626.22998 ]
 [1633.310059]
 [1658.810059]
 [1640.26001 ]
 [1614.369995]
 [1588.219971]
 [1591.      ]
 [1638.01001 ]
 [1640.      ]
 [1622.650024]
 [1607.949951]
 [1627.579956]
 [1622.099976]
 [1619.439941]
 [1631.560059]
 [1633.      ]
 [1636.400024]
 [1641.089966]
 [1639.829956]
 [1671.72998 ]
 [1696.170044]
 [1692.430054]
 [1668.949951]
 [1625.949951]
 [1620.800049]
 [1670.619995]
 [1673.099976]
 [1690.810059]
 [1686.219971]
 [1712.359985]
 [1742.150024]
 [1761.849976]
 [1797.27002 ]
 [1819.26001 ]
 [1764.77002 ]
 [1774.26001 ]
 [1783.76001 ]
 [1765.699951]
 [1773.420044]
 [1780.75    ]
 [1814.189941]
 [1813.97998 ]
 [1820.699

In [20]:
# Getting the predicted stock price from 12/03/2019 to 12/04/2019
#Part I: 
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Part 2:  We predict with our 'model_regressor' the output data  is scaled 
stock_price_prediction = model_regressor.predict(X_test)
#print (stock_price_prediction)

# Part 3: We use 'sc'from Scikit learning to retransform and unscaled the data 
stock_price_prediction = sc.inverse_transform(stock_price_prediction)
print (stock_price_prediction)


[[1761.4624]
 [1770.953 ]
 [1777.724 ]
 [1782.779 ]
 [1786.8822]
 [1792.1951]
 [1798.0206]
 [1804.2574]
 [1810.2657]
 [1816.8597]
 [1824.1887]
 [1830.6914]
 [1836.9264]
 [1842.3906]]


## Visualising the results

In [22]:
#plot and compare the prediction
training_set_1 = dataset.iloc[0:2285, 5:]
testing_set_1 = dataset.iloc [2285:, 5:]

trace_1 = go.Scatter(x = training_set_1.index, y=training_set, mode = 'lines', name ='Training')
trace_2 = go.Scatter(x = testing_set_1.index, y=testing_set, mode = 'lines', name = 'Testing')
trace_3 = go.Scatter(x = testing_set_1.index, y=stock_price_prediction, mode = 'lines', name = 'Prediction')

data = [trace_1, trace_2, trace_3]
layout = dict(title = 'LSTM Amazon Stock Market Prediction',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Adj Close'),
              )
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='LSTM')

In [23]:
# Visualising just the prediction

training_set_1 = dataset_f.iloc[0:1027]
testing_set_1 = dataset_f.iloc [1027:]


#plot and compare the predictions
trace_testing = go.Scatter(x = testing_set_1.index, y=testing_set, mode = 'lines', name = 'Real Amazon Stock Price')
trace_forecasting = go.Scatter(x = testing_set_1.index, y=stock_price_prediction, mode = 'lines', name = 'Predicted Amazon Stock Price')

data = [trace_testing, trace_forecasting]
layout = dict(title = 'LSTM Amazon Stock Market Prediction',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Adj Close'),
              )
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='LSTM')

In [21]:
rmse = sqrt(mean_squared_error(testing_set, stock_price_prediction))
print(rmse)

18.47422635054383
